In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier, plot_importance
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 14 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_ad_campaign_hit           38829 non-null float64
ad_campaign_hit                38829 non-null bool
cant_lead                      38829 non-null float64
lead                           38829 non-null bool
cant_brand_listing             38829 non-null float64
cant_visited_site              38829 non-null float64
cant_generic_listing           38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
dtypes: bool(2), float64(11), int64(1)
memory usage: 3.9+ MB


In [6]:
df2 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')


######################################################################################


df5 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')

df8 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('data/channel_device.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('data/check.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('data/view_sku.csv', low_memory=False, index_col='person')


df_unidos = df1.join(df2).fillna(0)
df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
#df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
#df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10).fillna(0)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)
df_unidos = df_unidos.join(df13).fillna(0)
df_unidos = df_unidos.join(df14).fillna(0)
df_unidos = df_unidos.join(df15).fillna(0)
df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 71 columns):
cant_conversions                                38829 non-null float64
cant_checkouts                                  38829 non-null float64
cant_viewed_product                             38829 non-null float64
cant_searched_product                           38829 non-null float64
cant_ad_campaign_hit                            38829 non-null float64
ad_campaign_hit                                 38829 non-null bool
cant_lead                                       38829 non-null float64
lead                                            38829 non-null bool
cant_brand_listing                              38829 non-null float64
cant_visited_site                               38829 non-null float64
cant_generic_listing                            38829 non-null float64
total_sesiones                                  38829 non-null int64
promedio_eventos_por_sesion               

In [7]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')


df_test = df_unidos.join(labels, how= "inner")
df_test.head()

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,cant_lead,lead,cant_brand_listing,cant_visited_site,...,device_type_Unknown,channel_Direct,channel_Email,channel_Organic,channel_Paid,channel_Referral,channel_Social,check_sku,sku,label
person,,,,,,,,,,,,,,,,,,,,,
ad93850f,0.0,1.0,20.0,0.0,10.0,True,0.0,False,15.0,5.0,...,0,0,0,0,0,0,0,0.0,5904.0,0
1b9f7cf6,0.0,1.0,9.0,8.0,0.0,False,0.0,False,3.0,3.0,...,0,0,0,0,0,0,0,0.0,9987.0,0
de8fe91b,0.0,1.0,27.0,13.0,0.0,False,0.0,False,3.0,2.0,...,0,0,0,0,0,0,0,0.0,10337.0,0
45baf068,0.0,1.0,6.0,0.0,5.0,True,0.0,False,3.0,1.0,...,0,0,0,0,0,0,0,0.0,3371.0,0
99abca5a,2.0,18.0,630.0,5.0,12.0,True,1.0,True,30.0,56.0,...,0,0,0,0,0,0,0,0.0,8567.0,0


In [8]:
X, y = df_test.iloc[:,:-1],df_test.iloc[:,-1]  #X van los features, en y el label, asumo que esta bien esto

In [9]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [11]:
#y_train

In [18]:
param_grid = { 
    'n_estimators': [x for x in range(200, 400, 100)],
    'learning_rate':[0.01],
    'max_depth' : [2,3,4],
    'gamma':[0.5,1,2],
    
    'scale_pos_weight': [x for x in range(2, 6, 1)]
} 

In [19]:
from sklearn.model_selection import GridSearchCV 
xgb = XGBClassifier(random_state=80)
CV_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv= 5, scoring='roc_auc', n_jobs=-1, verbose=10) 
CV_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.868273206188, total=  12.2s
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.821272130902, total=  12.5s
[CV]  n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.830647329944, total=  12.4s
[CV] n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV] n_estimators=200, scale_pos_weight=3, l

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   24.8s


[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.824959572553, total=  12.2s
[CV] n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.835023941598, total=  12.0s
[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.865879859629, total=  12.1s
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.819988784867, total=  12.1s
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.859944920569, total=  12.1s
[CV] n_estimators=200, scale_pos

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   37.7s


[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.825891070952, total=  12.1s
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.835575317536, total=  12.1s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.818165804686, total=  12.2s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, gamma=0.5, score=0.860756639525, total=  12.2s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=2, gamma=0.5 
[CV]  n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_d

...

[CV]  n_estimators=300, scale_pos_weight=3, learning_rate=0.01, max_depth=3, gamma=2, score=0.837166994532, total=  26.0s
[CV] n_estimators=300, scale_pos_weight=3, learning_rate=0.01, max_depth=3, gamma=2 
[CV]  n_estimators=300, scale_pos_weight=3, learning_rate=0.01, max_depth=3, gamma=2, score=0.871869998666, total=  26.0s
[CV] n_estimators=300, scale_pos_weight=4, learning_rate=0.01, max_depth=3, gamma=2 
[CV]  n_estimators=300, scale_pos_weight=3, learning_rate=0.01, max_depth=3, gamma=2, score=0.850378041688, total=  26.6s
[CV] n_estimators=300, scale_pos_weight=4, learning_rate=0.01, max_depth=3, gamma=2 
[CV]  n_estimators=300, scale_pos_weight=3, learning_rate=0.01, max_depth=3, gamma=2, score=0.833236399567, total=  27.9s
[CV] n_estimators=300, scale_pos_weight=4, learning_rate=0.01, max_depth=3, gamma=2 
[CV]  n_estimators=300, scale_pos_weight=3, learning_rate=0.01, max_depth=3, gamma=2, score=0.869778885239, total=  28.4s
[CV] n_estimators=300, scale_pos_weight=4, learnin

[Parallel(n_jobs=-1)]: Done 330 tasks      | elapsed: 29.1min


[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=4, gamma=2, score=0.834885136036, total=  18.7s
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=4, gamma=2 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=4, gamma=2, score=0.874064333822, total=  18.7s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=4, gamma=2 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=4, gamma=2, score=0.844930989412, total=  19.0s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=4, gamma=2 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=4, gamma=2, score=0.838590687271, total=  19.6s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=4, gamma=2 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=4, gamma=2, score=0.865587101429, total=  19.9s
[CV] n_estimators=200, scale_pos_weight=5, learnin

[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 32.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=80, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [200, 300], 'scale_pos_weight': [2, 3, 4, 5], 'learning_rate': [0.01], 'max_depth': [2, 3, 4], 'gamma': [0.5, 1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [21]:
CV_xgb.best_params_ #aca te dice el mejor hiperparametro


{'gamma': 0.5,
 'learning_rate': 0.01,
 'max_depth': 3,
 'n_estimators': 300,
 'scale_pos_weight': 4}

In [40]:
xg_reg = XGBClassifier(gamma = 0.5,max_depth=4,learning_rate=0.01,n_estimators=300,scale_pos_weight = 4,silent=True,n_jobs=-1)
#falta jugar con los hiperparametros
# Voting con multinomialNB, RF, y SVC (no lineal), probar las distintas combinaciones

In [41]:
xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.5, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=4,
       seed=None, silent=True, subsample=1)

In [42]:
score=xg_reg.score(X_test,y_test)*100
print(score)
#94.7205768736
#print(xgb.roc_auc_score(y_test,y_pred_proba))

92.4285346382


In [33]:
feature_importances = pd.DataFrame(xg_reg.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(40)

,importance
sum_timestamp,0.123908
ult_view,0.105747
timestamp,0.101839
prom_timestamp,0.067816
cant_checkouts,0.066437
min_timestamp,0.064138
cant_conversions,0.060230
cant_brand_listing,0.046437
suma_sku,0.040460
sku,0.040460


In [34]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')


kaggle_pred = xg_reg.predict_proba(df_events)
proba_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_comprar)
df_submit['label'] = series.values

df_submit.to_csv('submit_xg_2_1.csv')

In [35]:
len(df_events)

19415

In [36]:
df_submit["label"].mean()

0.16299859413988899